In [ ]:
from dotenv import load_dotenv
import pandas as pd
import os
import boto3
from botocore.exceptions import ClientError
import json
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter

data_folder = "../../data"
test_file_path = os.path.join(data_folder, "merged_instructions.csv")
df = pd.read_csv(test_file_path)
df['instruction'].head()

In [ ]:
def strip_code_fences(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    s = s.strip()
    if s.startswith("```"):
        body = s[3:]
        if "\n" in body:
            body = body.split("\n", 1)[1]
        if "```" in body:
            body = body.rsplit("```", 1)[0]
        return body.strip()
    return s

In [ ]:
def interpreter_result(code: str):
    # Configure and Start the code interpreter session
    code_client = CodeInterpreter('us-west-2')
    code_client.start()

    # Execute the hello world code
    response = code_client.invoke("executeCode", {
        "language": "python", 
        "code": code
    })

    # Capture the response instead of printing
    results = []
    for event in response["stream"]:
        results.append(event["result"])

    # Clean up and stop the code interpreter session 
    code_client.stop()

    # Example of using the captured data:
    # print(f"Captured {len(results)} result objects")
    # If you want to see the structure later:
    # print(json.dumps(results, indent=2))
    return results


def generate_code(instruction: str):
    client = boto3.client("bedrock-runtime", region_name="us-west-2")
    model_id = 'us.meta.llama3-2-11b-instruct-v1:0'

    prompt = f"You are a helpful assistant that generates code snippets based on user instructions." \
                "The method signature should be according to the given in the example part.\n" \
                "Use the following guidlines in generation: \n" \
                " - Provide the code in the response only\n" \
                " - Do not output any other texts or test cases results.\n" \
                " - Include the necessary import statements on the top even if they are usually not needed like - re, typing, itertools, Split etc. \n" \
                " - Do not include any comments in the code\n" \
                " - Do not forget to consider the edge cases\n" \
                " - Avoid recursion if possible\n" \
                " - Lastly, verify the code has no compilations error and contains the necessary imports.\n"
    
    # Set up messages and system message
    messages = [ { "role": "user", "content": [ { "text": instruction } ] } ]
    system = [ { "text": prompt } ]

    try:
        # Send the message to the model, using a basic inference configuration.
        response = client.converse(
            modelId=model_id,
            messages=messages,
            system=system,
            inferenceConfig={
                "maxTokens": 1024, 
                "temperature": 0.5
            },
        )

        for content_block in response["output"]["message"]["content"]:
            if content_block.get("text"):
                return content_block.get("text")
            
        return None

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")

In [ ]:

result_data = []

for index, row in df.iterrows():
    instruction = row['instruction']

    code = generate_code(instruction)
    code = strip_code_fences(code)
    execution_result = interpreter_result(code)

    retry_count = 0
    while execution_result[0]['isError'] and retry_count < 5:
        print(f"Retrying for id {row['id']}, attempt {retry_count + 1}")

        error_message = ""
        if "structuredContent" in execution_result[0]:
            error_message = execution_result[0]["structuredContent"]["stderr"]
        else:
            error_message = execution_result[0]["content"][0]["text"]

        retry_message = instruction + "\nCurrent code:\n" + code + "\nError Message:\n" + error_message + "\nPlease fix the code."
        code = generate_code(retry_message)
        code = strip_code_fences(code)
        execution_result = interpreter_result(code)
        retry_count += 1

    result_data.append({
        "id": row['id'],
        "response": code
    })
    print("Completed:", row['id'])
    
result_df = pd.DataFrame(result_data)

In [ ]:
# Create a json from the result_df
result_json = result_df.to_json(orient="records", indent=4)
# save json in the data_folder os.path.join(data_folder, "submission.json")
with open("llama_3.2_11B_submission.json", "w") as json_file:
    json_file.write(result_json)